In [0]:
from pyspark.sql.functions import *
from delta.tables import *

In [0]:
dbutils.widgets.text("silver_container", "silver")
dbutils.widgets.text("sgaccount", "sgonpremtoclouddev")

dbutils.widgets.text("target_schema", "gold")
dbutils.widgets.text("target_table", "CustomerOrderSummary")

# Read values passed from ADF
silver_container = dbutils.widgets.get("silver_container")
sgaccount        = dbutils.widgets.get("sgaccount")

target_schema    = dbutils.widgets.get("target_schema")
target_table     = dbutils.widgets.get("target_table")

# silver_container = 'silver'
# storage_account = 'sgonpremtoclouddev'
dfs_path = f"abfss://{silver_container}@{sgaccount}.dfs.core.windows.net/"

In [0]:
df_customers = spark.read.format("delta").load(f"{dfs_path}Customers")
df_orders = spark.read.format("delta").load(f"{dfs_path}Orders").filter(col("IsCurrent") == True)
df_orderitems = spark.read.format("delta").load(f"{dfs_path}OrderItems").filter("IsCurrent = true")
df_payments = spark.read.format("delta").load(f"{dfs_path}Payments")

In [0]:
df_orders_joined = (
    df_orders.alias("o")
    .join(df_orderitems.alias("oi"), col("o.OrderId")==col("oi.OrderID"), "left")
    .join(df_payments.alias("p"), col("o.OrderId")==col("p.OrderID"), "left")
)

In [0]:
df_customer_summary = (
    df_orders_joined
    .groupBy("o.CustomerID")
    .agg(
        countDistinct("o.OrderId").alias("TotalOrders"),
        sum("oi.TotalPrice").alias("TotalSpent"),
        round(avg("oi.TotalPrice"),2).alias("AvgOrderValue"),
        min("o.OrderDate").alias("FirstOrderDate"),
        max("o.OrderDate").alias("RecentOrderDate")
    )
)

In [0]:
df_customer_summary_final = (
    df_customers.select("CustomerID", "FirstName", "LastName","Email")
    .join(df_customer_summary, "CustomerID","left")
    .fillna({
        "TotalOrders": 0,
        "TotalSpent": 0.0,
        "AvgOrderValue": 0.0
    })
    .withColumn("LoadDate",current_timestamp())
)

In [0]:
# sql_server_name = "azsql-server-onprem-to-cloud.database.windows.net"  # or on-prem DNS
# sql_database    = "azdbonpremtocloud"
# sql_user        = "sqladmin"#dbutils.secrets.get("kv-scope", "sql-user")
# sql_password    = "Dev@1234"#dbutils.secrets.get("kv-scope", "sql-password")

sql_server_name = dbutils.secrets.get(scope="dbc-secret-scope-onprem-to-cloud", key="akv-azsql-server-name")
sql_database = dbutils.secrets.get(scope="dbc-secret-scope-onprem-to-cloud", key="akv-azsql-database")
sql_user = dbutils.secrets.get(scope="dbc-secret-scope-onprem-to-cloud", key="akv-azsql-username")
sql_password = dbutils.secrets.get(scope="dbc-secret-scope-onprem-to-cloud", key="akvs-azsql-password")

jdbc_url = (
    f"jdbc:sqlserver://{sql_server_name}:1433;"
    f"database={sql_database};"
    "encrypt=true;trustServerCertificate=false;"
    "hostNameInCertificate=*.database.windows.net;loginTimeout=30;"
)
sql_driver = "com.microsoft.sqlserver.jdbc.SQLServerDriver"

In [0]:
(
    df_customer_summary_final
        .write
        .format("jdbc")
        .option("url", jdbc_url)
        .option("dbtable", f"{target_schema}.{target_table}")
        .option("user", sql_user)
        .option("password", sql_password)
        .option("driver", sql_driver)
        .mode("overwrite")
        .save()
)